# **AI影像辨識 第6課 -- StarDist 細胞分割套件**

可用於辨識2D或3D細胞核的開源模型與套件

StarDist開源網址，與說明:  
https://github.com/stardist/stardist

ImageJ套件，安裝與說明:  
https://imagej.net/plugins/stardist

In [ ]:
#安裝套件
!pip install stardist
# Reinstall NumPy.
!pip install -U numpy==1.26.0
# Restart the runtime or kernel.

In [ ]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model 導入預訓練模型
model = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

img = test_image_nuclei_2d()

labels, _ = model.predict_instances(normalize(img))

plt.subplot(1,2,1)
plt.imshow(img, cmap="gray")
plt.axis("off")
plt.title("input image")

plt.subplot(1,2,2)
plt.imshow(render_label(labels, img=img))
plt.axis("off")
plt.title("prediction + input overlay")

# 以下示範如何使用stardist來處理youtube上的影片

In [ ]:
# 安裝模組
!pip install yt-dlp

In [ ]:
# 讀取 YouTube 影片
!yt-dlp https://youtu.be/lQLsyf64xak

In [ ]:
import cv2
import numpy as np
from stardist.models import StarDist2D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


# 載入 StarDist 模型
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# 設定參數
AI_cutoff_prob = 0.4

url = "https://youtu.be/lQLsyf64xak"

# 開啟下載影片
cap = cv2.VideoCapture('/content/Zebrafish Embryo Growing Its Elaborate Sensory Nervous System ｜ 2018 Nikon Small World [lQLsyf64xak].webm')

# 取得影片尺寸
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 建立 VideoWriter 物件
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (width, height))

# 處理每一幀
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        # 將彩色影像轉換為灰階影像
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 使用 StarDist 模型進行細胞核辨識
        labels, details = model.predict_instances(normalize(gray))

        # 標記細胞核
        for i in range(len(details['points'])):
            if (details['prob'][i] > AI_cutoff_prob):
                cv2.circle(frame, (int(details['points'][i][1]), int(details['points'][i][0])), 5, (0, 255, 0), -1)

        # 將處理後的幀寫入輸出影片
        out.write(frame)

        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# 釋放資源
cap.release()
out.release()
cv2.destroyAllWindows()